In [ ]:
import re

import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
from apache_beam.runners.interactive.interactive_beam import *


class ReadWordsFromText(beam.PTransform):
    
    def __init__(self, file_pattern):
        self._file_pattern = file_pattern
    
    def expand(self, pcoll):
        return pcoll.pipeline | beam.io.ReadFromText(self._file_pattern) | beam.FlatMap(lambda line: re.findall(r'[\w\']+', line.strip(), re.UNICODE))

In [ ]:
p = beam.Pipeline(InteractiveRunner())
words = p | 'read' >> ReadWordsFromText('gs://apache-beam-samples/shakespeare/kinglear.txt')
show(words)

In [ ]:
counts = (words 
          | 'count' >> beam.combiners.Count.PerElement()
          | 'dict' >> beam.MapTuple(lambda word, count: {'word': word, 'count': count}))
show(counts)

In [ ]:
output = counts | 'format' >> beam.Map(lambda d: '%s: %d' % (d['word'], d['count']))
show(output)

In [ ]:
output | 'write' >> beam.io.WriteToText('/tmp/output.txt')

In [ ]:
p.run()

In [ ]:
# Give output a new format.
output = counts | 'format' >> beam.Map(lambda d: '%s -> %d' % (d['word'], d['count']))
show(output)

In [ ]:
# Also output to new targets.
output | 'write to local' >> beam.io.WriteToText('/tmp/count.txt')
output | 'write to gcs' >> beam.io.WriteToText('gs://ib_notebook_examples/simply_write/count.txt')

In [ ]:
p.run()